# ChimeraX Color Script Generator Demo

This notebook demonstrates how to use `analysis.chimerax_color_lib` to map
per-residue scalar values (e.g. entropy) to a ChimeraX `.cxc` color script.

In [ ]:
import numpy as np
from analysis.chimerax_color_lib import generate_chimerax_script, write_chimerax_script

## 1. Create sample per-residue entropy values

Replace this with your real data (e.g. loaded from a CSV or computed via `entropy_lib`).

In [ ]:
np.random.seed(42)
num_residues = 200
entropy_values = np.random.exponential(scale=0.5, size=num_residues)
print(f"Residues: {num_residues}")
print(f"Entropy range: [{entropy_values.min():.3f}, {entropy_values.max():.3f}]")

## 2. Generate a ChimeraX script with color mapping

In [ ]:
script = generate_chimerax_script(
    entropy_values,
    cmap_name="Greys",
    transform_method="quantile",
    color=True,
    color_invert=False,
    transparency=False,
)

# Preview the first 10 lines
for line in script.splitlines()[:10]:
    print(line)

## 3. Add transparency mapping

In [ ]:
script_with_transparency = generate_chimerax_script(
    entropy_values,
    cmap_name="viridis",
    transform_method="none",
    color=True,
    transparency=True,
    transparency_invert=True,
)

# Preview first and last few lines
lines = script_with_transparency.splitlines()
for line in lines[:5]:
    print(line)
print("...")
for line in lines[-5:]:
    print(line)

## 4. Write the script to a `.cxc` file

In [ ]:
output_path = write_chimerax_script(script, "sample_data/demo_color_output.cxc")
print(f"Written to: {output_path}")